# Construcción de un corpus

La realización del corpus se hará sobre las conferencias de prensa ("mañaneras") de la presidenta Claudia Sheinbaum y del ex-presidente Andres Manuel López Obrador.  

**Fuentes**
* Para las conferencias de la presidenta: https://www.gob.mx/presidencia/es/archivo/articulos?fechaFin=2025-01-23+00%3A00%3A00&fechaInicio=2024-11-01+00%3A00%3A00&idiom=es&order=DESC&q=Conferencia+de+prensa&section=articulos&site=presidencia&utf8=%E2%9C%93&page=1

* Para las conferencias del ex-presidente: https://amlo.presidente.gob.mx/sala-de-prensa/transcripciones/page/1


## Preparación

#### **Importamos primero las principales librerías que utilizaremos**

In [6]:
import wget
import glob
import re
from bs4 import BeautifulSoup
import shutil
import datetime
import os

#### Creamos carpetas en caso de no existir.

In [63]:
carpetas = ["clean_estenograficas",
            "estenograficas",
            "paginas",
            "raw_discursos",
            "text_discursos"]

for carpeta in carpetas:
    if not os.path.isdir(f".\\{carpeta}\\"):
        os.mkdir(f".\\{carpeta}")

#### Creamos una función para verificar que un archivo no exista.

In [8]:
def no_existe(path):
    return not os.path.isfile(path)

## Para la presidenta Claudia Sheinbaum.

#### **Bajamos todas las páginas del archivo de mañaneras.**

In [64]:
url_pagina_i = r"https://www.gob.mx/presidencia/es/archivo/articulos?fechaFin=2025-01-23" + \
    r"+00%3A00%3A00&fechaInicio=2024-11-01+00%3A00%3A00&idiom=es&order=DESC"+\
        r"&q=Conferencia+de+prensa&section=articulos&site=presidencia&utf8=%E2%9C%93&page="

# Al 23 de Enero de 2025
num_paginas = 6

for i in range(1,num_paginas+1):
    url_pagina = url_pagina_i + str(i)
    nombre = 'paginas\\shein_pagina'+str(i)+'.txt'
    if no_existe(nombre):
        wget.download(url_pagina,nombre)

#### **Obtenemos las mañaneras en cada página y las descargamos**
El resultado se guarda en la carpeta `raw_discursos`.

In [65]:
urls_conf = []

# Expresion regular para encontrar links a las versiones estenográficas
# dentro de cada página.
patron = r'href="([^"]*version-estenografica[^"]*)"'

for f_pagina in glob.glob("./paginas/shein*.txt"):
    urls_conf += [
        path
        for path in re.findall(
            patron, open(f_pagina, "r", encoding="utf-8").read()
        )
        if "estenografica" in path
    ]

In [66]:
print("No. Mañaneras: ", len(urls_conf))
print(urls_conf[0])

No. Mañaneras:  52
/presidencia/es/articulos/version-estenografica-conferencia-de-prensa-de-la-presidenta-claudia-sheinbaum-pardo-del-23-de-enero-de-2025?idiom=es


In [67]:
for path in urls_conf:
    nombre = "./raw_discursos/"+path.split("/")[-1][:-10]+".txt"
    if no_existe(nombre):
        f = wget.download(url = "https://www.gob.mx"+path, out=nombre)

#### **Ahora limpiamos los html para obtener el puro texto de las páginas.**
El resultado se guarda en la carpeta `text_discursos`.

In [68]:
for f_pagina in glob.glob(".\\raw_discursos\\*.txt"):
    soup = BeautifulSoup(open(f_pagina, "r", encoding="utf-8").read(), "html.parser")
    filename = ".\\text_discursos\\"+f_pagina.replace(f".\\raw_discursos\\","")
    if no_existe(filename):
        open(filename, "w", encoding="utf-8").write(soup.get_text())

#### **Renombramos con la fecha**: 
* Al inicio de cada texto viene la fecha en formato ``"dd de \<mes\> de yyyy"``.  
    - El resultado se guarda en la carpeta `estenograficas`.

Le pondré que inicie con ``"|"`` para que encuentre solo una ocurrencia por archivo (Linea 196 de cada texto).  
Podría también, para cada búsqueda, simplemente seleccionar el primer elemento que encuentre, no sé cuál sea una mejor práctica.

In [69]:
fecha_re = r'\| \d{1,2} de [a-z]+ de \d{4}'

In [70]:
meses = {
    "enero" : 1,
    "febrero" : 2,
    "marzo" : 3,
    "abril" : 4,
    "mayo" : 5,
    "junio" : 6,
    "julio" : 7,
    "agosto" : 8,
    "septiembre" : 9,
    "octubre" : 10,
    "noviembre" : 11,
    "diciembre" : 12
}

In [71]:
# Verifico que encuentre 51 fechas, correspondiente a 51 mañaneras.
fecha = []

for f_pagina in glob.glob(".\\text_discursos\\*.txt"):
    fecha += re.findall(fecha_re, open(f_pagina, "r", encoding="utf-8").read())

print(len(fecha))
print(len(set(fecha)))
fecha[0][2:]

52
52


'10 de diciembre de 2024'

In [72]:
# Renombramos todos los archivos y los pasamos a la carpeta "estenograficas".
for f_pagina in glob.glob(".\\text_discursos\\*.txt"):
    # Extraer la fecha del archivo
    fecha = re.findall(fecha_re, open(f_pagina, "r", encoding="utf-8").read())[0][2:]
    fecha = fecha.split(" de ")
    fecha = datetime.date(int(fecha[2]), meses[fecha[1]], int(fecha[0]))
    
    # Nuevo nombre del archivo
    new_str = fecha.strftime("%Y-%m-%d")
    new_name = ".\\estenograficas\\" + new_str + ".txt"
        
    # Copiar el archivo
    if no_existe(new_name):
        shutil.copy(f_pagina, new_name)

#### **Limpiar textos:** 
* Hacer que contenga únicamente el texto relevante.
    - Se guardan en la carpeta `clean_estenograficas`

In [73]:
# Indicadores de inicio y final de la conferencia.

re_inicio = r"CLAUDIA SHEINBAUM PARDO:" # Siempre inician con la presidenta hablando.

re_final1 = r"—000—"
re_final2 = r"---000---"
re_final3 = r"---00---" # Terminan de 3 formas, está extraño.

In [74]:
for archivo in glob.glob(".\\estenograficas\\*.txt"):
    actual_archivo = open(archivo, "r", encoding="utf-8")
    lineas = actual_archivo.readlines()
    nuevo_nombre = archivo.replace("estenograficas","clean_estenograficas")
    nuevo_archivo = open(nuevo_nombre, "w")
    
    inicio = False
    for linea in lineas:
        # Definimos el inicio
        if (bool(re.search(re_inicio,linea))):
            inicio = True
        if not inicio:
            continue
        
        # Definimos el final
        if (bool(re.search(re_final1, linea)) or
            bool(re.search(re_final2, linea)) or
            bool(re.search(re_final3, linea))):
            break
        
        # En caso de que ya sea el cuerpo del texto.
        nuevo_archivo.write(linea)
    nuevo_archivo.close()
    actual_archivo.close()
    

## Borrar archivos ya innecesarios.

* Eliminar el contenido de las carpetas `raw_discursos`, `paginas` y `text_discursos`.

In [75]:
carpetas = ["raw_discursos",
            "paginas",
            "text_discursos",
            "estenograficas"]

for carpeta in carpetas:
    for archivo in glob.glob(f".\\{carpeta}\\*.txt"):
        os.remove(archivo)

## Para el ex-presidente López Obrador.

In [46]:
url_pagina_i = r"https://amlo.presidente.gob.mx/sala-de-prensa/transcripciones/page/"

num_paginas = 287

for i in range(1,num_paginas+1):
    url_pagina = url_pagina_i + str(i)
    nombre = './paginas/amlo_pagina'+str(i)+'.txt'
    if no_existe(nombre):
        wget.download(url_pagina,nombre)

In [47]:
urls_conf = []

# Expresion regular para encontrar links a las versiones estenográficas
# dentro de cada página.
patron = r'href="([^"]*conferencia-de-prensa[^"]*)"'

for f_pagina in glob.glob("./paginas/amlo*.txt"):
    urls_conf += [
        path
        for path in re.findall(
            patron, open(f_pagina, "r", encoding="utf-8").read()
        )
        if "estenografica" in path
    ]
    
urls_conf = [url for url in urls_conf if not url.endswith("#comments")]
urls_conf = sorted(list(set(urls_conf)))

In [50]:
print("No. Mañaneras: ", len(urls_conf))
print(urls_conf[0].split("/")[-2])
print(urls_conf[0])

No. Mañaneras:  1370
01-02-22-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador
https://amlo.presidente.gob.mx/01-02-22-version-estenografica-de-la-conferencia-de-prensa-matutina-del-presidente-andres-manuel-lopez-obrador/


In [51]:
for path in urls_conf:
    nombre = path.split("/")[-2]
    nombre_archivo = "./raw_discursos/"+nombre+".txt"
    if no_existe(nombre_archivo):
        f = wget.download(url = path, out=nombre_archivo)

In [52]:
for f_pagina in glob.glob(".\\raw_discursos\\*.txt"):
    soup = BeautifulSoup(open(f_pagina, "r", encoding="utf-8").read(), "html.parser")
    filename = ".\\text_discursos\\"+f_pagina.replace(f".\\raw_discursos\\","")
    if no_existe(filename):
        open(filename, "w", encoding="utf-8").write(soup.get_text())

In [53]:
fecha_re = r'[a-z] \d{2}.\d{2}.\d{2}'

In [ ]:
# Verifico que encuentre 1370 fechas, correspondiente a 1370 mañaneras.
fechas = []

for f_pagina in glob.glob(".\\text_discursos\\*.txt"):
    fechas_en_archivo = re.findall(fecha_re, open(f_pagina, "r", encoding="utf-8").read())
    
    if not fechas_en_archivo:
        # Si no encuentra fechas, elimina el documento.
        print("No coincidio en:", f_pagina)
        os.remove(f_pagina)
        continue
    
    if fechas_en_archivo[0] in fechas:
        # Simplemente quitaremos mañaneras con segunda parte, etc.
        # Son pocas (8).
        print("Fecha repetida: ",fechas_en_archivo[0], f_pagina)
        os.remove(f_pagina)
        continue

    fechas.append(fechas_en_archivo[0]) 
    

print(len(fechas))
print(len(set(fechas)))
fechas = [fecha[2:] for fecha in fechas]
fechas[:3]

1359
1359


['01.02.22', '01.02.23', '01.03.21']

In [56]:
# Renombramos todos los archivos y los pasamos a la carpeta "estenograficas".
for f_pagina in glob.glob(".\\text_discursos\\*.txt"):
    
    # Extraer la fecha del archivo
    fecha = re.findall(fecha_re, open(f_pagina, "r", encoding="utf-8").read())[0][2:]
    fecha = fecha.split(".")
    fecha = datetime.date(2000 + int(fecha[2]), int(fecha[1]), int(fecha[0]))
    
    # Nuevo nombre del archivo
    new_str = fecha.strftime("%Y-%m-%d")
    new_name = ".\\estenograficas\\" + new_str + ".txt"
        
    # Copiar el archivo
    if no_existe(new_name):
        shutil.copy(f_pagina, new_name)

In [57]:
# Indicadores de inicio y final de la conferencia.

re_inicio = r"ANDRÉS MANUEL LÓPEZ OBRADOR:" # Siempre inician con la presidenta hablando.

# Terminan de mil formas, no me voy a esforzar en esto

In [58]:
for archivo in glob.glob(".\\estenograficas\\*.txt"):
    actual_archivo = open(archivo, "r", encoding="utf-8")
    lineas = actual_archivo.readlines()
    nuevo_nombre = archivo.replace("estenograficas","clean_estenograficas")
    nuevo_archivo = open(nuevo_nombre, "w")
    
    inicio = False
    for linea in lineas:
        # Definimos el inicio
        if (bool(re.search(re_inicio,linea))):
            inicio = True
        if not inicio:
            continue
        
        # En caso de que ya sea el cuerpo del texto.
        nuevo_archivo.write(linea)
    nuevo_archivo.close()
    actual_archivo.close()
    

## Borrar archivos ya innecesarios.

* Eliminar el contenido de las carpetas `raw_discursos`, `paginas` y `text_discursos`.

In [59]:
carpetas = ["raw_discursos",
            "paginas",
            "text_discursos",
            "estenograficas"]

for carpeta in carpetas:
    for archivo in glob.glob(f".\\{carpeta}\\*.txt"):
        os.remove(archivo)
    os.rmdir(f".\\{carpeta}")

## Generamos el corpus

In [94]:
# Generamos el archivo para corpus
corpus = ""
for archivo in glob.glob(".\\clean_estenograficas\\*.txt"):
    corpus += open(archivo, "r", encoding="utf-8").read() + "\n"

with open(".\\corpus.txt", "w", encoding="utf-8") as archivo:
    archivo.write(corpus)